In [1]:
import numpy as np

from model import HiFiGAN, MelSpectrogram

In [2]:
model = HiFiGAN(training=False)
mel_spectrogram = MelSpectrogram()


In [3]:
import torch
cp = torch.load('../generator.pth', map_location='cpu')
model.generator.load_state_dict(cp)

<All keys matched successfully>

In [4]:
def generate_audio(mel):
    with torch.no_grad():
        audio = model(mel)
    return audio.squeeze().numpy()

def preprocess_audio(audio, sr=None):
    if isinstance(audio, np.ndarray):
        audio = torch.from_numpy(audio)
    if audio.dim() == 1:
        audio = audio.unsqueeze(0)
    mel = mel_spectrogram(audio.float(), sr=sr)
    return mel

In [5]:
from datasets import load_dataset
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation", trust_remote_code=True)
sample = ds[0]["audio"]
sample_rate, waveform = sample['sampling_rate'], sample['array']

In [6]:
import IPython.display as ipd
ipd.Audio(waveform, rate=sample_rate)

In [7]:
torch.from_numpy(waveform).unsqueeze(0).shape

torch.Size([1, 93680])

In [8]:
full_audio = []
for i in range(0, len(waveform), 16000):
    audio = generate_audio(preprocess_audio(waveform[i:i+16000]))
    full_audio.extend(audio)

In [9]:
ipd.Audio(full_audio, rate=sample_rate)

In [10]:
import torch
import torchaudio
waveform, sample_rate = torchaudio.load("../data/1170331873109626940.ogg")
waveform = waveform[0]
ipd.Audio(waveform, rate=sample_rate)

In [11]:
waveform = torch.nn.functional.pad(waveform, (0, 16000 - waveform.shape[0] % 16000))
waveform = waveform.reshape(-1, 16000)
preprocessed = preprocess_audio(waveform, sr=sample_rate)
generated = generate_audio(preprocessed).reshape(-1)
ipd.Audio(generated, rate=sample_rate)

min value is  tensor(-1.0775)
max value is  tensor(1.1709)
